In [13]:
!"C:\Program Files\Wireshark\tshark.exe" -v


TShark (Wireshark) 4.6.2 (v4.6.2-0-g24d5e2b5a3dc).

Copyright 1998-2025 Gerald Combs <gerald@wireshark.org> and contributors.
Licensed under the terms of the GNU General Public License (version 2 or later).
This is free software; see the file named COPYING in the distribution. There is
NO WARRANTY; not even for MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.

Compile-time info:
 Bit width: 64-bit
  Compiler: Microsoft Visual Studio 2022 (VC++ 14.44, build 35221)
      GLib: 2.84.2
 With:
  +brotli                     +MaxMind
  +Gcrypt 1.11.2-unknown      +nghttp2 1.65.0
  +GnuTLS 3.8.11 and PKCS#11  +nghttp3 1.8.0
  +Kerberos (MIT)             +PCRE2 10.45 2025-02-05
  +libpcap                    +Snappy 1.1.9
  +libsmi 0.5.0               +xxhash 0.8.3
  +libxml2 2.13.8             +zlib 1.3.1
  +Lua 5.4.6 (UfW patched)    +zlib-ng 2.2.3
  +LZ4 1.10.0                 +Zstandard 1.5.7

Runtime info:
      OS: 64-bit Windows 11 (24H2), build 26100
     CPU: AMD Ryzen 5 5500U with 

In [16]:
!cmd /c ""C:\Program Files\Wireshark\tshark.exe" -r data/capture.pcapng -T fields -e ip.src -e ip.dst > ip_output.txt"


In [18]:
import os
os.path.getsize("ip_output.txt")


3244265

In [19]:
import ipaddress

public_ips = set()

with open("ip_output.txt") as f:
    for line in f:
        for ip in line.strip().split('\t'):
            if ip:
                try:
                    if not ipaddress.ip_address(ip).is_private:
                        public_ips.add(ip)
                except ValueError:
                    pass

print("Total public IPs:", len(public_ips))
list(public_ips)[:10]


Total public IPs: 94


['98.87.105.159',
 '98.94.240.7',
 '54.192.142.68',
 '74.125.200.132',
 '150.171.27.10',
 '52.182.143.214',
 '172.64.155.209',
 '74.125.130.119',
 '103.65.28.109',
 '224.0.0.1']

# Geolocate IPs

In [21]:
import requests
import pandas as pd
import time

geo_data = []

for ip in list(public_ips)[:50]:  # limit for API safety
    try:
        r = requests.get(f"http://ip-api.com/json/{ip}", timeout=5).json()
        if r["status"] == "success":
            geo_data.append({
                "ip": ip,
                "lat": r["lat"],
                "lon": r["lon"],
                "country": r["country"],
                "city": r["city"]
            })
        time.sleep(1)  # avoid rate limit
    except:
        pass

df = pd.DataFrame(geo_data)
df.head()


,ip,lat,lon,country,city
0,98.87.105.159,39.0438,-77.4874,United States,Ashburn
1,98.94.240.7,39.0438,-77.4874,United States,Ashburn
2,54.192.142.68,28.6139,77.2088,India,New Delhi
3,74.125.200.132,37.4225,-122.0850,United States,Mountain View
4,150.171.27.10,45.5019,-73.5674,Canada,Montreal


In [25]:
import folium

# 1️⃣ Create the base map FIRST
world_map = folium.Map(location=[20, 0], zoom_start=2)

# 2️⃣ Define color rules
color_map = {
    "United States": "red",
    "India": "green",
    "Canada": "blue"
}

# 3️⃣ Add markers to the map
for _, row in df.iterrows():
    folium.CircleMarker(
        location=[row["lat"], row["lon"]],
        radius=6,
        popup=f"{row['ip']} → {row['city']}, {row['country']}",
        color=color_map.get(row["country"], "purple"),
        fill=True,
        fill_opacity=0.7
    ).add_to(world_map)

# 4️⃣ Display the map (LAST line of cell)
world_map


# Identify HIGH-RISK COUNTRIES (Threat Intelligence Angle)

In [26]:
country_counts = df["country"].value_counts()
country_counts


country
United States    21
India            17
Canada            8
Singapore         1
Name: count, dtype: int64

# Detect SUSPICIOUS IP BEHAVIOR (SOC Skill)

In [28]:
ip_frequency = df["ip"].value_counts()
ip_frequency.head(10)


ip
98.87.105.159     1
98.94.240.7       1
54.192.142.68     1
74.125.200.132    1
150.171.27.10     1
52.182.143.214    1
172.64.155.209    1
74.125.130.119    1
103.65.28.109     1
34.226.235.51     1
Name: count, dtype: int64

# Tagging CLOUD PROVIDERS

In [29]:
def classify_provider(city, country):
    if country == "United States" and city in ["Ashburn", "Mountain View"]:
        return "Cloud / CDN"
    return "Other"

df["provider_type"] = df.apply(
    lambda x: classify_provider(x["city"], x["country"]),
    axis=1
)

df["provider_type"].value_counts()


provider_type
Other          30
Cloud / CDN    17
Name: count, dtype: int64

# SECURITY DASHBOARD VIEW

In [30]:
summary = {
    "Total External IPs": df["ip"].nunique(),
    "Countries Reached": df["country"].nunique(),
    "Top Country": df["country"].value_counts().idxmax(),
    "Cloud Traffic %": (
        df["provider_type"].value_counts(normalize=True).get("Cloud / CDN", 0) * 100
    )
}

summary


{'Total External IPs': 47,
 'Countries Reached': 4,
 'Top Country': 'United States',
 'Cloud Traffic %': np.float64(36.17021276595745)}